NOMBRES: Diego Alberto

APELLIDOS: Leiva Pérez

CARNE: 21752

FECHA: 04/09/2025

## Librerias

In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter

## Utilidades para Texto

In [2]:
def normalize_text(text: str) -> str:
    """
    Convierte a minúsculas, elimina puntuación y caracteres especiales,
    y normaliza espacios en blanco.
    Args:
        text (str): Texto a normalizar.
    Returns:
        str: Texto normalizado.
    """
    text = text.lower()
    # Conserva letras (incluye tildes/ñ), dígitos y espacios.
    text = re.sub(r"[^a-záéíóúüñ0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize(text: str):
    """
    Tokeniza el texto normalizado en palabras.
    Args:
        text (str): Texto a tokenizar.
    Returns:
        list: Lista de tokens (palabras).
    """
    return normalize_text(text).split()

def build_vocab(tokenized_docs, min_freq: int = 1):
    """
    Construye el vocabulario a partir de documentos tokenizados.
    Args:
        tokenized_docs (list): Lista de documentos tokenizados.
        min_freq (int): Frecuencia mínima para incluir una palabra en el vocabulario.
    Returns:
        tuple: (vocabulario, word2idx, idx2word, counts)
    """
    counts = Counter([t for doc in tokenized_docs for t in doc])
    vocab = [w for w, c in counts.items() if c >= min_freq]
    vocab = sorted(vocab)
    word2idx = {w: i for i, w in enumerate(vocab)}
    idx2word = {i: w for i, w in enumerate(vocab)}
    return vocab, word2idx, idx2word, counts

def sliding_windows_indices(center, window_size, n):
    """
    Obtiene los índices de las palabras en una ventana deslizante alrededor de una palabra central.
    Args:
        center (int): Índice de la palabra central.
        window_size (int): Tamaño de la ventana (número de palabras a cada lado).
        n (int): Longitud total de la secuencia.
    Returns:
        list: Lista de índices en la ventana deslizante.
    """
    start = max(0, center - window_size)
    end = min(n, center + window_size + 1)
    return [i for i in range(start, end) if i != center]

## Ejercicio 1
Cree un corpus a su gusto como el visto en clase, cálcule PPMI, pero aplicando Lapace Smoothing.

In [3]:
corpus_docs = [
    # Abuelos/as y edad/actividad
    "el abuelo mayor juega ajedrez en la sala con la abuela mayor",
    "la abuela mayor cuida el jardin y hace jardineria con el abuelo mayor",
    # Padres y madres + actividades
    "el padre adulto repara autos en el taller y aprende mecanica",
    "la madre adulta estudia medicina y cuida la casa con paciencia",
    # Hijos/as + actividades/edad
    "el hijo joven practica skate en el parque despues de la escuela",
    "la hija joven pinta cuadros y aprende pintura en la escuela",
    # Tios/as + actividades/edad
    "el tio adulto sale a pesca y conversa en la sala",
    "la tia adulta baila danza y organiza reuniones en la casa",
    # Primos/as + actividades/edad
    "el primo joven juega videojuegos y estudia con el hijo joven",
    "la prima joven hace fotografia y practica pintura con la hija joven",
    # Sobrinos/as + edad
    "el sobrino niño visita la casa y lee cuentos en la sala",
    "la sobrina niña dibuja en el jardin y juega con la prima joven",
    # Conexiones cruzadas (refuerza co-ocurrencias)
    "el padre adulto conversa con el abuelo mayor en la sala",
    "la madre adulta camina con la abuela mayor en el jardin",
    "el tio adulto repara herramientas en el taller con el padre adulto",
    "la tia adulta organiza danza en la escuela y ayuda a la madre adulta",
    "el primo joven patina skate en el parque y habla con el sobrino niño",
    "la prima joven toma fotografia en el jardin y pinta con la hija joven",
    "el hijo joven visita la casa y estudia en la escuela con el primo joven",
    "la hija joven practica pintura en la sala y conversa con la sobrina niña",
]

# Preprocesamiento
tokenized_docs = [tokenize(d) for d in corpus_docs] # Lista de listas de tokens
vocab, word2idx, idx2word, counts = build_vocab(tokenized_docs, min_freq=1) # Vocabulario y mapeos

# Estadísticas básicas
print("Docs:", len(tokenized_docs)) # Número de documentos
print("Vocab size:", len(vocab)) # Tamaño del vocabulario 
print("Top-10 más frecuentes:", Counter([t for d in tokenized_docs for t in d]).most_common(10)) # Top-10 palabras más frecuentes

Docs: 20
Vocab size: 68
Top-10 más frecuentes: [('la', 30), ('el', 24), ('en', 16), ('y', 15), ('con', 13), ('joven', 13), ('mayor', 6), ('sala', 5), ('adulto', 5), ('adulta', 5)]


## Ejercicio 2

Crear 10 analigias validas basadas en su texto, tome en cuenta que es necesario utilizar el codigo visto en clase y ajuste su corpus para poder lograr obtener todas las analogias